---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [5]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [51]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [73]:
def answer_one():
    
    print(df.loc[:,"Gold"].sort_values(ascending = False)[:5])
    most_gold_medals = df.loc[:,"Gold"].argmax()
         
    return most_gold_medals

answer_one()

United States    976
Soviet Union     395
Great Britain    236
France           202
China            201
Name: Gold, dtype: int64


'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [71]:
def answer_two():
    
    summer_gold_counts = df.loc[:,"Gold"]
    winter_gold_counts = df.loc[:,"Gold.1"]
    
    summer_winter_difference = abs(summer_gold_counts - winter_gold_counts)
    print(summer_winter_difference.sort_values(ascending = False)[:5])   
    biggest_diff = summer_winter_difference.argmax()
    
    return biggest_diff

answer_two()

United States    880
Soviet Union     317
Great Britain    226
China            189
France           171
dtype: int64


'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [21]:
def answer_three():
    
    #only return countries with at least 1 gold in summer and winter
    summer_gold_count = df.loc[(df["Gold"] >=1) & (df["Gold.1"] >=1),"Gold"] 
    winter_gold_count = df.loc[(df["Gold"] >=1) & (df["Gold.1"] >=1),"Gold.1"]
        
    total_gold_count = (summer_gold_count + winter_gold_count)
    
    percent_difference_summer_winter = ((summer_gold_count - winter_gold_count) 
                                        / total_gold_count)
    print(percent_difference_summer_winter.sort_values(ascending = False)[:5])
    
    answer = percent_difference_summer_winter.argmax()
    
    return answer

answer_three()

Bulgaria          0.961538
Spain             0.947368
Belgium           0.947368
Australia         0.930556
Czechoslovakia    0.921569
dtype: float64


'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [82]:
def answer_four():
    
    Points = pd.Series(df["Gold.2"]*3 + df["Silver.2"]*2 + df["Bronze.2"]*1)
    Points = Points.sort_values(ascending = False)
    
    return Points

answer_four()

United States             5684
Soviet Union              2526
Great Britain             1574
Germany                   1546
France                    1500
Italy                     1333
Sweden                    1217
China                     1120
East Germany              1068
Russia                    1042
Norway                     985
Hungary                    962
Australia                  923
Finland                    895
Japan                      866
Canada                     846
Netherlands                727
Switzerland                630
South Korea                609
Romania                    572
Austria                    569
Poland                     520
West Germany               459
Cuba                       420
Bulgaria                   411
Denmark                    335
Czechoslovakia             327
Unified Team               287
Belgium                    276
United Team of Germany     269
                          ... 
Grenada                      3
Haiti   

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [6]:
census_df = pd.read_csv('census.csv')
census_df.head()


,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [105]:
def answer_five():
    
    #create multiindex
    census_df.set_index(['STNAME','CTYNAME'],drop = False, inplace = True)

    #SUMLEV 50 = County, SUMLEV 40 = State. Return only rows with SUMLEV == 50.
    census_df2 = census_df.loc[census_df["SUMLEV"] == 50,:]
    
    #Group dataframe by state and return the number of counties by state
    groupby_stname_counts = census_df2.groupby(["STNAME"]).count()
             
    print(groupby_stname_counts.iloc[:,0].sort_values(ascending=False)[:5])
    
    most_counties = groupby_stname_counts.iloc[:,0].argmax()
          
    return most_counties

answer_five()

STNAME
Texas       254
Georgia     159
Virginia    133
Kentucky    120
Missouri    115
Name: SUMLEV, dtype: int64


'Texas'


### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [23]:
def answer_six():
    
      
    census_df.set_index(['STNAME','CTYNAME'],drop = False, inplace = True) #create multiindex   
    census_df2 = census_df.loc[census_df["SUMLEV"] == 50,:] 
    #SUMLEV 50 = County, SUMLEV 40 = State. Return only rows with SUMLEV == 50.
        
    census_df2["CENSUS2010POP_Rank"] = (census_df2.groupby(["STNAME"])["CENSUS2010POP"]
                                        .rank(ascending=False))
    #create a new series with each county's population rank.
    census_df3 = census_df2.loc[census_df2["CENSUS2010POP_Rank"] <= 3,:] 
    #filter rows with rank in the top 3
    
    sum_top_3_counties_census2010POP = census_df3.groupby(["STNAME"])["CENSUS2010POP"].sum() 
    #sum the population for the top 3 counties by each state
            
    print(sum_top_3_counties_census2010POP.sort_values(ascending=False)[:5])
    
    top_3_states = list((sum_top_3_counties_census2010POP
                        .sort_values(ascending=False)[:3]
                        .index))
    #return the index (state name) for the 3 most populous states
    
    return top_3_states

answer_six()

STNAME
California    15924150
Texas          8269632
Illinois       6815061
New York       6321295
Florida        5564635
Name: CENSUS2010POP, dtype: int64


['California', 'Texas', 'Illinois']

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [26]:
def answer_seven():
    
    census_df.set_index(['STNAME','CTYNAME'],drop = False, inplace = True) 
    #create multi-index of state name and city name
    census_df2 = census_df.loc[census_df["SUMLEV"] == 50,:] 
    #boolean mask to create a new df with only county data
    
    for i in range(11,16):
        census_df2["POPCHG"+str(i)] = abs(census_df2["POPESTIMATE20"+str(i)]
                                          -census_df2["POPESTIMATE20"+str(i-1)]) 
        #create 5 new columns of data for the abs. value difference in pop year over year.
               
    census_df2["MAXPOPCHG"] = (census_df2[["POPCHG11","POPCHG12","POPCHG13",
                                           "POPCHG14","POPCHG15"]]
                               .max(axis = 1)) 
    #create a new column with the maximum of the population change columns
    
    print(census_df2["MAXPOPCHG"].sort_values(ascending=False)[:5])
    
    answer = census_df2["MAXPOPCHG"].argmax()[1]
    #return the county with the largest absolute population change (2010-2015)
       
    return answer

answer_seven()

STNAME      CTYNAME           
Texas       Harris County         95115
Arizona     Maricopa County       77925
California  Los Angeles County    74739
Florida     Miami-Dade County     71906
Texas       Dallas County         46666
Name: MAXPOPCHG, dtype: int64


'Harris County'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [28]:
def answer_eight():
       
    census_df.reset_index(drop = True, inplace = True)
    
    census_df2 = census_df.loc[(census_df["SUMLEV"] == 50) 
                                & (census_df["CTYNAME"].str.startswith("Washington"))
                                & (census_df["POPESTIMATE2014"] 
                                   < census_df["POPESTIMATE2015"])
                               ,:]
    #only regions 1 & 2
    #county level data only
    #county name starts with Washington
    census_df3 = census_df2.loc[(census_df2["REGION"] ==1) | (census_df2["REGION"] ==2)]
    #2014 population is less than 2015 population
    
    print(census_df3.loc[:,["SUMLEV","CTYNAME","POPESTIMATE2014","POPESTIMATE2015","REGION"]])
    st_name_cty_name = census_df3[["STNAME","CTYNAME"]]
    
    return st_name_cty_name

answer_eight()

      SUMLEV            CTYNAME  POPESTIMATE2014  POPESTIMATE2015  REGION
896       50  Washington County            22087            22247       2
1419      50  Washington County           249320           251597       2
2345      50  Washington County           208175           208261       1
2355      50  Washington County           126430           126517       1
3163      50  Washington County           133301           133674       2


,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
